In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import selenium
import json

In [3]:
from dataclasses import dataclass
@dataclass
class Team:
    name: str
    abbrev: str
    wins: int = 0
    losses: int = 0
    elo: float = 1500

In [5]:
from datetime import datetime, timedelta
'''
    Get game results and elo from schedule
'''

team_dict = {}

SCOREBOARD_URL = "https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?limit=1000&dates=20231106-20240314"

res = requests.get(SCOREBOARD_URL)
espn_games_dict = res.json()

for event in espn_games_dict['events']:
    away_team, home_team = re.split(r' (?:@|VS) ', event['shortName'])
    away_name, home_name = event['name'].split(" at ")
    
    if away_team not in team_dict: team_dict[away_team] = Team(away_name, away_team)
    if home_team not in team_dict: team_dict[home_team] = Team(home_name, home_team)
    
    # TODO create some kind of travel pentalty here
    
    score_home = 1 if event['competitions'][0]['competitors'][0] else 0
    score_away = int(not(score_home))
    
    q_home = 10**(team_dict[home_team].elo/400)
    q_away = 10**(team_dict[away_team].elo/400)
    
    e_home = q_home/(q_home+q_away)
    e_away = q_away/(q_home+q_away)
    
    K = 24
    
    team_dict[home_team].elo += K * (score_home - e_home)
    team_dict[away_team].elo += K * (score_away - e_away)
    
    if score_home == 1: # home team won
        team_dict[home_team].wins += 1
        team_dict[away_team].losses += 1
    else:
        team_dict[home_team].losses += 1
        team_dict[away_team].wins += 1
    
team_df = pd.DataFrame(team_dict.values())
team_df.sort_values(by=['elo'], ascending=False, ignore_index=True).to_csv('ncaa_teams_elo.csv')


# SCHEDULE_URL = "https://www.espn.com/mens-college-basketball/schedule/_/date/20231106"


In [44]:

# computer 
bpi_tables = pd.read_html("https://www.espn.com/mens-college-basketball/bpi")
# print(bpi_tables[0][''])
bpi_tables[1]['Team'] = bpi_tables[0]['Team']
bpi_tables[1]['CONF'] = bpi_tables[0]['CONF']

bpi_tables[1].to_csv('bpi.csv')
# human rankings
# sagarin_table = pd.read_html("http://www.usatoday.com/sports/ncaab/sagarin/")
# ncaa_table = pd.read_html("")


In [22]:
# from bs4 import BeautifulSoup

pom_table = pd.read_html("pomeroy.html")
print(pom_table.head(20))

[     Unnamed: 0_level_0      Unnamed: 1_level_0  Unnamed: 2_level_0  \
                     Rk                    Team                Conf   
     Unnamed: 0_level_2      Unnamed: 1_level_2  Unnamed: 2_level_2   
                     Rk                    Team                Conf   
     Unnamed: 0_level_4      Unnamed: 1_level_4  Unnamed: 2_level_4   
                     Rk                    Team                Conf   
     Unnamed: 0_level_6      Unnamed: 1_level_6  Unnamed: 2_level_6   
                     Rk                    Team                Conf   
     Unnamed: 0_level_8      Unnamed: 1_level_8  Unnamed: 2_level_8   
                     Rk                    Team                Conf   
    Unnamed: 0_level_10     Unnamed: 1_level_10 Unnamed: 2_level_10   
                     Rk                    Team                Conf   
    Unnamed: 0_level_12     Unnamed: 1_level_12 Unnamed: 2_level_12   
                     Rk                    Team                Conf   
    U

In [26]:
lrmc_table = pd.read_html("lrmc.html")
print(lrmc_table)

[     Unnamed: 0  Through games of 3/11/2024 Through games of 3/11/2024.1  \
0           NaN                   LRMC Rank                         Team   
1           NaN                         NaN                          NaN   
2           NaN                           1                      Arizona   
3           NaN                           2                      Houston   
4           NaN                           3                    Tennessee   
..          ...                         ...                          ...   
431         NaN  Through games of 3/11/2024   Through games of 3/11/2024   
432         NaN                   LRMC Rank                         Team   
433         NaN                         NaN                          NaN   
434         NaN                         361                    Coppin_St   
435         NaN                         362                 MS_Valley_St   

    Through games of 3/11/2024.2  Unnamed: 4  \
0                     Conference      

In [29]:
moore_table = pd.read_csv("moore.csv")
print(moore_table)

      RK                    TEAM   W   L  T    SOS     PR
0      1                 HOUSTON  28   3  0  79.19  95.51
1      2             CONNECTICUT  28   3  0  77.50  93.63
2      3                  PURDUE  28   3  0  79.82  92.32
3      4                 ARIZONA  24   7  0  78.57  92.26
4      5                  AUBURN  24   7  0  77.69  91.82
..   ...                     ...  ..  .. ..    ...    ...
357  358              COPPIN ST.   2  26  0  66.35  52.18
358  359                 PACIFIC   6  26  0  64.64  51.64
359  360                     VMI   4  28  0  64.63  51.20
360  361                   IUPUI   6  26  0  63.43  50.69
361  362  MISSISSIPPI VALLEY ST.   1  30  0  67.77  49.01

[362 rows x 7 columns]
